In [ ]:
!pip install adversarial-robustness-toolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import cv2
import os
import pickle
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import time
from math import log10, sqrt
from torch.utils.data import DataLoader
from art.utils import load_cifar10
import random
from torch.quantization import MovingAverageMinMaxObserver
from torch.ao.quantization.observer import MinMaxObserver
from torch.quantization import QuantStub, DeQuantStub
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [ ]:
def test(model: nn.Module, dataloader: DataLoader, cuda=False) -> float:
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            if cuda:
              inputs = inputs.cuda()
              labels = labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def predict(img):
  img = torch.from_numpy(img)
  img = F.normalize(img, [0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
  return img.numpy()

def custom_collate(batch):
        # Combine a list of samples into a batch
        data, labels = zip(*batch)
        data = torch.stack(data)
        labels = torch.tensor(labels, dtype=torch.long)
        return data, labels

def evaluator(model, loader):
  model.eval()
  top_1 = 0
  top_5 = 0
  with torch.no_grad():
    for data in loader:
      inputs, labels = data
      outputs = model(inputs)

      _, predicted = torch.max(outputs, 1, keepdim=True)
      top_1 += torch.sum(predicted.view(-1) == labels).item()

      _, predicted_5 = torch.topk(outputs, k=5, dim=1)
      top_5 += torch.sum(predicted_5 == labels.unsqueeze(1)).item()

  return ("{:.2f}".format((top_1/400) * 100), "{:.2f}".format((top_5/400) * 100))

def evaluator_testset(model, loader):
  model.eval()
  top_1 = 0
  top_5 = 0
  with torch.no_grad():
    for data in loader:
      inputs, labels = data
      outputs = model(inputs)

      _, predicted = torch.max(outputs, 1, keepdim=True)
      top_1 += torch.sum(predicted.view(-1) == labels).item()

      _, predicted_5 = torch.topk(outputs, k=5, dim=1)
      top_5 += torch.sum(predicted_5 == labels.unsqueeze(1)).item()

  return ("{:.2f}".format((top_1/10000) * 100), "{:.2f}".format((top_5/10000) * 100))



transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,shuffle=False, num_workers=2, pin_memory=True)

with open("/content/drive/MyDrive/Thesis/FashionMnist_set/testset.pkl", 'rb') as f:
  fashion_testset = pickle.load(f)

fashion_testloader = torch.utils.data.DataLoader(fashion_testset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)


100%|██████████| 170498071/170498071 [00:02<00:00, 71751710.48it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_cifar10()

x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)

classes_cifar = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
classes_fashion = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle-boot']


with open("/content/drive/MyDrive/Thesis/FashionMnist_set/fashionmnist_label.pkl",'rb') as f:
  y_f_train_set = pickle.load(f)

with open("/content/drive/MyDrive/Thesis/FashionMnist_set/fashionmnist_test_label.pkl",'rb') as f:
  y_f_test_set = pickle.load(f)

y_f_train_set = y_f_train_set[0:400]

y_f_test_set = y_f_test_set[0:400]

y_test_set = np.zeros((400,),np.int8)

y_train_set = np.zeros((400,),np.int8)


for i in range(400):
        y_test_set[i] = np.where(y_test[i] == 1)[0][0]

for i in range(400):
        y_train_set[i] = np.where(y_train[i] == 1)[0][0]

## CIFAR-10 ResNet Quantized Model Definition

In [ ]:
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url
from functools import partial
from typing import Dict, Type, Any, Callable, Union, List, Optional
from torch.ao.nn.quantized.modules.functional_modules import FloatFunctional
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride
        self.ff = torch.nn.quantized.FloatFunctional()
    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        #out += identity
        out = self.ff.add(out, identity)
        out = self.relu(out)

        return out


class CifarResNet(nn.Module):

    def __init__(self, block, layers, num_classes=10):
        super(CifarResNet, self).__init__()
        self.inplanes = 16
        self.conv1 = conv3x3(3, 16)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(block, 16, layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x



## CIFAR-10 MobileNet Quantized Model Definition

In [ ]:
from functools import partial
from typing import Dict, Type, Any, Callable, Union, List, Optional
from torch import Tensor
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url
from torch.ao.nn.quantized.modules.functional_modules import FloatFunctional

def _make_divisible(v: float, divisor: int, min_value: Optional[int] = None) -> int:
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        f_add = torch.nn.quantized.FloatFunctional()
        new_v = f_add.add(new_v, divisor)
        #new_v += divisor
    return new_v


class ConvBNActivation(nn.Sequential):
    def __init__(
        self,
        in_planes: int,
        out_planes: int,
        kernel_size: int = 3,
        stride: int = 1,
        groups: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        activation_layer: Optional[Callable[..., nn.Module]] = None,
        dilation: int = 1,
    ) -> None:
        padding = (kernel_size - 1) // 2 * dilation
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if activation_layer is None:
            activation_layer = nn.ReLU6
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, dilation=dilation, groups=groups,
                      bias=False),
            norm_layer(out_planes),
            activation_layer(inplace=True)
        )
        self.out_channels = out_planes


# necessary for backwards compatibility
ConvBNReLU = ConvBNActivation

class InvertedResidual(nn.Module):
    def __init__(
        self,
        inp: int,
        oup: int,
        stride: int,
        expand_ratio: int,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        hidden_dim = int(round(inp * expand_ratio))
        self.use_res_connect = self.stride == 1 and inp == oup

        layers: List[nn.Module] = []
        if expand_ratio != 1:
            # pw
            layers.append(ConvBNReLU(inp, hidden_dim, kernel_size=1, norm_layer=norm_layer))
        layers.extend([
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, stride=stride, groups=hidden_dim, norm_layer=norm_layer),
            # pw-linear
            nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
            norm_layer(oup),
        ])
        self.conv = nn.Sequential(*layers)
        self.out_channels = oup
        self._is_cn = stride > 1
        self.ff = torch.nn.quantized.FloatFunctional()

    def forward(self, x: Tensor) -> Tensor:
        if self.use_res_connect:
            #return x + self.conv(x)
            return self.ff.add(x, self.conv(x))
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(
        self,
        num_classes: int = 10,
        width_mult: float = 1.0,
        inverted_residual_setting: Optional[List[List[int]]] = None,
        round_nearest: int = 8,
        block: Optional[Callable[..., nn.Module]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        """
        MobileNet V2 main class

        Args:
            num_classes (int): Number of classes
            width_mult (float): Width multiplier - adjusts number of channels in each layer by this amount
            inverted_residual_setting: Network structure
            round_nearest (int): Round the number of channels in each layer to be a multiple of this number
            Set to 1 to turn off rounding
            block: Module specifying inverted residual building block for mobilenet
            norm_layer: Module specifying the normalization layer to use

        """
        super(MobileNetV2, self).__init__()

        if block is None:
            block = InvertedResidual

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        input_channel = 32
        last_channel = 1280

        if inverted_residual_setting is None:
            inverted_residual_setting = [
                # t, c, n, s
                [1, 16, 1, 1],
                [6, 24, 2, 1],  # NOTE: change stride 2 -> 1 for CIFAR10/100
                [6, 32, 3, 2],
                [6, 64, 4, 2],
                [6, 96, 3, 1],
                [6, 160, 3, 2],
                [6, 320, 1, 1],
            ]

        # only check the first element, assuming user knows t,c,n,s are required
        if len(inverted_residual_setting) == 0 or len(inverted_residual_setting[0]) != 4:
            raise ValueError("inverted_residual_setting should be non-empty "
                             "or a 4-element list, got {}".format(inverted_residual_setting))

        # building first layer
        input_channel = _make_divisible(input_channel * width_mult, round_nearest)
        self.last_channel = _make_divisible(last_channel * max(1.0, width_mult), round_nearest)
        features: List[nn.Module] = [ConvBNReLU(3, input_channel, stride=1, norm_layer=norm_layer)]  # NOTE: change stride 2 -> 1 for CIFAR10/100
        # building inverted residual blocks
        for t, c, n, s in inverted_residual_setting:
            output_channel = _make_divisible(c * width_mult, round_nearest)
            for i in range(n):
                stride = s if i == 0 else 1
                features.append(block(input_channel, output_channel, stride, expand_ratio=t, norm_layer=norm_layer))
                input_channel = output_channel
        # building last several layers
        features.append(ConvBNReLU(input_channel, self.last_channel, kernel_size=1, norm_layer=norm_layer))
        # make it nn.Sequential
        self.features = nn.Sequential(*features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.last_channel, num_classes),
        )

        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # This exists since TorchScript doesn't support inheritance, so the superclass method
        # (this one) needs to have a name other than `forward` that can be accessed in a subclass
        x = self.features(x)
        # Cannot use "squeeze" as batch-size can be 1
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


## Model Quantization

In [ ]:
resnet20_fp_model = CifarResNet(BasicBlock, [3]*3)

resnet20_fp_model.load_state_dict(torch.load("/content/drive/MyDrive/Thesis/ResNet20/CIFAR-10/Train_iteration_3/Models/3rd_Iteration_retrained_model_quant_91.35acc.pkl",map_location=torch.device('cpu')))

resnet20_model = CifarResNet(BasicBlock, [3]*3)

resnet20_model.load_state_dict(torch.load("/content/drive/MyDrive/Thesis/ResNet20/CIFAR-10/Train_iteration_3/Models/3rd_Iteration_retrained_model_quant_91.35acc.pkl",map_location=torch.device('cpu')))

resnet20_model = torch.quantization.QuantWrapper(resnet20_model)
B=8
resnet20_model.qconfig = torch.quantization.QConfig(activation= MovingAverageMinMaxObserver.with_args(quant_min=0, quant_max=int(2 ** B - 1), dtype=torch.quint8,
                                                              qscheme=torch.per_tensor_affine, reduce_range=False),
                                                     weight= MovingAverageMinMaxObserver.with_args(quant_min=int(-(2 ** B) / 2), quant_max=int((2 ** B) / 2 - 1), dtype=torch.qint8,
                                                              qscheme=torch.per_tensor_symmetric, reduce_range=False))
torch.quantization.prepare(resnet20_model, inplace=True)

resnet20_model.to("cpu")
test(resnet20_model, testloader, cuda=False)
resnet20_model.to("cpu")

torch.quantization.convert(resnet20_model, inplace=True)


QuantWrapper(
  (quant): Quantize(scale=tensor([0.0203]), zero_point=tensor([120]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (module): CifarResNet(
    (conv1): QuantizedConv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.05107611045241356, zero_point=126, padding=(1, 1), bias=False)
    (bn1): QuantizedBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): QuantizedConv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.014130653813481331, zero_point=169, padding=(1, 1), bias=False)
        (bn1): QuantizedBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): QuantizedConv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.010054154321551323, zero_point=122, padding=(1, 1), bias=False)
        (bn2): QuantizedBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_runn

In [ ]:
resnet56_fp_model = CifarResNet(BasicBlock,[9]*3)

resnet56_fp_model.load_state_dict(torch.load("/content/drive/Thesis/Thesis/ResNet56/CIFAR-10/Train Iteration 3/model/ResNet56_3it_CIFAR10_92.33acc.pkl",map_location=torch.device('cpu')))

resnet56_model = CifarResNet(BasicBlock,[9]*3)

resnet56_model.load_state_dict(torch.load("/content/drive/Thesis/Thesis/ResNet56/CIFAR-10/Train Iteration 3/model/ResNet56_3it_CIFAR10_92.33acc.pkl",map_location=torch.device('cpu')))

resnet56_model = torch.quantization.QuantWrapper(resnet56_model)
B=8
resnet56_model.qconfig = torch.quantization.QConfig(activation= MovingAverageMinMaxObserver.with_args(quant_min=0, quant_max=int(2 ** B - 1), dtype=torch.quint8,
                                                              qscheme=torch.per_tensor_affine, reduce_range=False),
                                                     weight= MovingAverageMinMaxObserver.with_args(quant_min=int(-(2 ** B) / 2), quant_max=int((2 ** B) / 2 - 1), dtype=torch.qint8,
                                                              qscheme=torch.per_tensor_symmetric, reduce_range=False))
torch.quantization.prepare(resnet56_model, inplace=True)

resnet56_model.to("cpu")
test(resnet56_model, testloader, cuda=False)
resnet56_model.to("cpu")

torch.quantization.convert(resnet56_model, inplace=True)

QuantWrapper(
  (quant): Quantize(scale=tensor([0.0203]), zero_point=tensor([120]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (module): CifarResNet(
    (conv1): QuantizedConv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.03820133954286575, zero_point=125, padding=(1, 1), bias=False)
    (bn1): QuantizedBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): QuantizedConv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.012166980654001236, zero_point=161, padding=(1, 1), bias=False)
        (bn1): QuantizedBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): QuantizedConv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.006007997784763575, zero_point=136, padding=(1, 1), bias=False)
        (bn2): QuantizedBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_runn

In [ ]:
mobilenet_fp_model = MobileNetV2(width_mult = 1.4)

mobilenet_fp_model.load_state_dict(torch.load("/content/drive/MyDrive/Thesis/MobileNetv2_x1_4/CIFAR-10/Train Iteration 3/model/MobileNet_3it_CIFAR10_92.79acc.pkl",map_location=torch.device('cpu')))


mobilenet_model = MobileNetV2(width_mult = 1.4)

mobilenet_model.load_state_dict(torch.load("/content/drive/MyDrive/Thesis/MobileNetv2_x1_4/CIFAR-10/Train Iteration 3/model/MobileNet_3it_CIFAR10_92.79acc.pkl",map_location=torch.device('cpu')))

mobilenet_model = torch.quantization.QuantWrapper(mobilenet_model)
B=8
mobilenet_model.qconfig = torch.quantization.QConfig(activation= MovingAverageMinMaxObserver.with_args(quant_min=0, quant_max=int(2 ** B - 1), dtype=torch.quint8,
                                                              qscheme=torch.per_tensor_affine, reduce_range=False),
                                                     weight= MovingAverageMinMaxObserver.with_args(quant_min=int(-(2 ** B) / 2), quant_max=int((2 ** B) / 2 - 1), dtype=torch.qint8,
                                                              qscheme=torch.per_tensor_symmetric, reduce_range=False))
torch.quantization.prepare(mobilenet_model, inplace=True)

mobilenet_model.to("cpu")
test(mobilenet_model, testloader, cuda=False)
mobilenet_model.to("cpu")

torch.quantization.convert(mobilenet_model, inplace=True)

/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/utils.py:317: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


QuantWrapper(
  (quant): Quantize(scale=tensor([0.0203]), zero_point=tensor([120]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (module): MobileNetV2(
    (features): Sequential(
      (0): ConvBNActivation(
        (0): QuantizedConv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), scale=0.0380074605345726, zero_point=120, padding=(1, 1), bias=False)
        (1): QuantizedBatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): QuantizedReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNActivation(
            (0): QuantizedConv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), scale=0.006269583944231272, zero_point=109, padding=(1, 1), groups=48, bias=False)
            (1): QuantizedBatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): QuantizedReLU6(inplace=True)
          )
          (1): QuantizedConv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), scale=0.

In [ ]:
# ResNet20 DataLoaders

test_20_0 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_20_test_0)).type(torch.FloatTensor), torch.from_numpy(y_test_set).type(torch.LongTensor))
test_20_0_loader = torch.utils.data.DataLoader(dataset=test_20_0, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_20_1 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_20_test_1)).type(torch.FloatTensor), torch.from_numpy(y_test_set).type(torch.LongTensor))
test_20_1_loader = torch.utils.data.DataLoader(dataset=test_20_1, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_20_2 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_20_test_2)).type(torch.FloatTensor), torch.from_numpy(y_test_set).type(torch.LongTensor))
test_20_2_loader = torch.utils.data.DataLoader(dataset=test_20_2, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_20_3 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_20_test_3)).type(torch.FloatTensor), torch.from_numpy(y_test_set).type(torch.LongTensor))
test_20_3_loader = torch.utils.data.DataLoader(dataset=test_20_3, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)


train_20_0 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_20_train_0)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))
train_20_0_loader = torch.utils.data.DataLoader(dataset=train_20_0, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_20_1 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_20_train_1)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))
train_20_1_loader = torch.utils.data.DataLoader(dataset=train_20_1, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_20_2 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_20_train_2)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))
train_20_2_loader = torch.utils.data.DataLoader(dataset=train_20_2, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_20_3 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_20_train_3)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))
train_20_3_loader = torch.utils.data.DataLoader(dataset=train_20_3, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)



In [ ]:
# ResNet56 DataLoaders

test_56_0 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_56_test_0)).type(torch.FloatTensor), torch.from_numpy(y_test_set).type(torch.LongTensor))
test_56_0_loader = torch.utils.data.DataLoader(dataset=test_56_0, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_56_1 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_56_test_1)).type(torch.FloatTensor), torch.from_numpy(y_test_set).type(torch.LongTensor))
test_56_1_loader = torch.utils.data.DataLoader(dataset=test_56_1, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_56_2 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_56_test_2)).type(torch.FloatTensor), torch.from_numpy(y_test_set).type(torch.LongTensor))
test_56_2_loader = torch.utils.data.DataLoader(dataset=test_56_2, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_56_3 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_56_test_3)).type(torch.FloatTensor), torch.from_numpy(y_test_set).type(torch.LongTensor))
test_56_3_loader = torch.utils.data.DataLoader(dataset=test_56_3, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)


train_56_0 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_56_train_0)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))
train_56_0_loader = torch.utils.data.DataLoader(dataset=train_56_0, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_56_1 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_56_train_1)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))
train_56_1_loader = torch.utils.data.DataLoader(dataset=train_56_1, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_56_2 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_56_train_2)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))
train_56_2_loader = torch.utils.data.DataLoader(dataset=train_56_2, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_56_3 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_56_train_3)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))
train_56_3_loader = torch.utils.data.DataLoader(dataset=train_56_3, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)



In [ ]:
#MobileNet Dataset

test_mob_0 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_mob_test_0)).type(torch.FloatTensor), torch.from_numpy(y_test_set).type(torch.LongTensor))
test_mob_0_loader = torch.utils.data.DataLoader(dataset=test_mob_0, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_mob_1 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_mob_test_1)).type(torch.FloatTensor), torch.from_numpy(y_test_set).type(torch.LongTensor))
test_mob_1_loader = torch.utils.data.DataLoader(dataset=test_mob_1, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_mob_2 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_mob_test_2)).type(torch.FloatTensor), torch.from_numpy(y_test_set).type(torch.LongTensor))
test_mob_2_loader = torch.utils.data.DataLoader(dataset=test_mob_2, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_mob_3 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_mob_test_3)).type(torch.FloatTensor), torch.from_numpy(y_test_set).type(torch.LongTensor))
test_mob_3_loader = torch.utils.data.DataLoader(dataset=test_mob_3, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)


train_mob_0 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_mob_train_0)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))
train_mob_0_loader = torch.utils.data.DataLoader(dataset=train_mob_0, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_mob_1 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_mob_train_1)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))
train_mob_1_loader = torch.utils.data.DataLoader(dataset=train_mob_1, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_mob_2 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_mob_train_2)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))
train_mob_2_loader = torch.utils.data.DataLoader(dataset=train_mob_2, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_mob_3 = torch.utils.data.TensorDataset(torch.from_numpy(predict(x_mob_train_3)).type(torch.FloatTensor), torch.from_numpy(y_train_set).type(torch.LongTensor))
train_mob_3_loader = torch.utils.data.DataLoader(dataset=train_mob_3, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

In [ ]:
print("CIFAR-10 TEST SETS:")


res20_1, res20_5 = evaluator(resnet20_model,test_20_0_loader)
print("ResNet20 0: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

res20_1, res20_5 = evaluator(resnet20_model,test_20_1_loader)
print("ResNet20 1: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

res20_1, res20_5 = evaluator(resnet20_model,test_20_2_loader)
print("ResNet20 2: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

res20_1, res20_5 = evaluator(resnet20_model,test_20_3_loader)
print("ResNet20 3: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

print("\n")

res56_1, res56_5 = evaluator(resnet56_model,test_56_0_loader)
print("ResNet56 0: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

res56_1, res56_5 = evaluator(resnet56_model,test_56_1_loader)
print("ResNet56 1: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

res56_1, res56_5 = evaluator(resnet56_model,test_56_2_loader)
print("ResNet56 2: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

res56_1, res56_5 = evaluator(resnet56_model,test_56_3_loader)
print("ResNet56 3: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

print("\n")

mob_1, mob_5 = evaluator(mobilenet_model,test_mob_0_loader)
print("MobileNet 0: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))
mob_1, mob_5 = evaluator(mobilenet_model,test_mob_1_loader)
print("MobileNet 1: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))
mob_1, mob_5 = evaluator(mobilenet_model,test_mob_2_loader)
print("MobileNet 2: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))
mob_1, mob_5 = evaluator(mobilenet_model,test_mob_3_loader)
print("MobileNet 3: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))

CIFAR-10 TEST SETS:
ResNet20 0: Top_1: 90.75% Top_5:99.75%
ResNet20 1: Top_1: 90.25% Top_5:100.00%
ResNet20 2: Top_1: 88.00% Top_5:100.00%
ResNet20 3: Top_1: 73.00% Top_5:99.75%


ResNet56 0: Top_1: 92.50% Top_5:99.75%
ResNet56 1: Top_1: 92.00% Top_5:99.75%
ResNet56 2: Top_1: 91.25% Top_5:99.75%
ResNet56 3: Top_1: 73.00% Top_5:99.75%


MobileNet 0: Top_1: 92.25% Top_5:100.00%
MobileNet 1: Top_1: 92.50% Top_5:100.00%
MobileNet 2: Top_1: 89.75% Top_5:99.50%
MobileNet 3: Top_1: 76.25% Top_5:99.25%


In [ ]:
print("CIFAR-10 TRAIN SETS:")


res20_1, res20_5 = evaluator(resnet20_model,train_20_0_loader)
print("ResNet20 0: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

res20_1, res20_5 = evaluator(resnet20_model,train_20_1_loader)
print("ResNet20 1: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

res20_1, res20_5 = evaluator(resnet20_model,train_20_2_loader)
print("ResNet20 2: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

res20_1, res20_5 = evaluator(resnet20_model,train_20_3_loader)
print("ResNet20 3: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

print("\n")

res56_1, res56_5 = evaluator(resnet56_model,train_56_0_loader)
print("ResNet56 0: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

res56_1, res56_5 = evaluator(resnet56_model,train_56_1_loader)
print("ResNet56 1: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

res56_1, res56_5 = evaluator(resnet56_model,train_56_2_loader)
print("ResNet56 2: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

res56_1, res56_5 = evaluator(resnet56_model,train_56_3_loader)
print("ResNet56 3: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

print("\n")

mob_1, mob_5 = evaluator(mobilenet_model,train_mob_0_loader)
print("MobileNet 0: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))
mob_1, mob_5 = evaluator(mobilenet_model,train_mob_1_loader)
print("MobileNet 1: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))
mob_1, mob_5 = evaluator(mobilenet_model,train_mob_2_loader)
print("MobileNet 2: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))
mob_1, mob_5 = evaluator(mobilenet_model,train_mob_3_loader)
print("MobileNet 3: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))

CIFAR-10 TRAIN SETS:
ResNet20 0: Top_1: 100.00% Top_5:100.00%
ResNet20 1: Top_1: 99.25% Top_5:100.00%
ResNet20 2: Top_1: 99.50% Top_5:100.00%
ResNet20 3: Top_1: 81.50% Top_5:100.00%


ResNet56 0: Top_1: 100.00% Top_5:100.00%
ResNet56 1: Top_1: 100.00% Top_5:100.00%
ResNet56 2: Top_1: 98.50% Top_5:100.00%
ResNet56 3: Top_1: 79.25% Top_5:100.00%


MobileNet 0: Top_1: 100.00% Top_5:100.00%
MobileNet 1: Top_1: 100.00% Top_5:100.00%
MobileNet 2: Top_1: 99.25% Top_5:100.00%
MobileNet 3: Top_1: 46.00% Top_5:75.25%


In [ ]:
print("CIFAR-10 TESTSET EVALUATOR")

res20_top1, res20_top5 = evaluator_testset(resnet20_fp_model, testloader)
print("ResNet20 Floating-Point Evaluation: Top 1: {}% Top 5: {}%".format(res20_top1, res20_top5))

res20_top1, res20_top5 = evaluator_testset(resnet20_model, testloader)
print("ResNet20 Quantized Evaluation: Top 1: {}% Top 5: {}%".format(res20_top1, res20_top5))


res56_top1, res56_top5 = evaluator_testset(resnet56_fp_model, testloader)
print("ResNet56 Floating-Point Evaluation: Top 1: {}% Top 5: {}%".format(res56_top1, res56_top5))

res56_top1, res56_top5 = evaluator_testset(resnet56_model, testloader)
print("ResNet56 Quantized Evaluation: Top 1: {}% Top 5: {}%".format(res56_top1, res56_top5))


mob_top1, mob_top5 = evaluator_testset(mobilenet_fp_model, testloader)
print("MobileNet Floating-Point Evaluation: Top 1: {}% Top 5: {}%".format(mob_top1, mob_top5))

mob_top1, mob_top5 = evaluator_testset(mobilenet_model, testloader)
print("MobileNet Quantized Evaluation: Top 1: {}% Top 5: {}%".format(mob_top1, mob_top5))


CIFAR-10 TESTSET EVALUATOR
ResNet20 Floating-Point Evaluation: Top 1: 91.35% Top 5: 99.56%
ResNet20 Quantized Evaluation: Top 1: 91.28% Top 5: 99.58%
ResNet56 Floating-Point Evaluation: Top 1: 92.33% Top 5: 99.67%
ResNet56 Quantized Evaluation: Top 1: 92.24% Top 5: 99.62%
MobileNet Floating-Point Evaluation: Top 1: 92.79% Top 5: 99.77%
MobileNet Quantized Evaluation: Top 1: 92.37% Top 5: 99.77%


## FashionMNIST ResNet Quantized Model Definition

In [ ]:
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url
from functools import partial
from typing import Dict, Type, Any, Callable, Union, List, Optional
from torch.ao.nn.quantized.modules.functional_modules import FloatFunctional
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride
        self.ff = torch.nn.quantized.FloatFunctional()
    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        #out += identity
        out = self.ff.add(out, identity)
        out = self.relu(out)

        return out


class CifarResNet(nn.Module):

    def __init__(self, block, layers, num_classes=10):
        super(CifarResNet, self).__init__()
        self.inplanes = 16
        self.conv1 = conv3x3(1, 16)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(block, 16, layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x



## FashionMNIST MobileNet Quantized Model Definition

In [ ]:
from functools import partial
from typing import Dict, Type, Any, Callable, Union, List, Optional
from torch import Tensor
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url
from torch.ao.nn.quantized.modules.functional_modules import FloatFunctional

def _make_divisible(v: float, divisor: int, min_value: Optional[int] = None) -> int:
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        f_add = torch.nn.quantized.FloatFunctional()
        new_v = f_add.add(new_v, divisor)
        #new_v += divisor
    return new_v


class ConvBNActivation(nn.Sequential):
    def __init__(
        self,
        in_planes: int,
        out_planes: int,
        kernel_size: int = 3,
        stride: int = 1,
        groups: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        activation_layer: Optional[Callable[..., nn.Module]] = None,
        dilation: int = 1,
    ) -> None:
        padding = (kernel_size - 1) // 2 * dilation
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if activation_layer is None:
            activation_layer = nn.ReLU6
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, dilation=dilation, groups=groups,
                      bias=False),
            norm_layer(out_planes),
            activation_layer(inplace=True)
        )
        self.out_channels = out_planes


# necessary for backwards compatibility
ConvBNReLU = ConvBNActivation

class InvertedResidual(nn.Module):
    def __init__(
        self,
        inp: int,
        oup: int,
        stride: int,
        expand_ratio: int,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        hidden_dim = int(round(inp * expand_ratio))
        self.use_res_connect = self.stride == 1 and inp == oup

        layers: List[nn.Module] = []
        if expand_ratio != 1:
            # pw
            layers.append(ConvBNReLU(inp, hidden_dim, kernel_size=1, norm_layer=norm_layer))
        layers.extend([
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, stride=stride, groups=hidden_dim, norm_layer=norm_layer),
            # pw-linear
            nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
            norm_layer(oup),
        ])
        self.conv = nn.Sequential(*layers)
        self.out_channels = oup
        self._is_cn = stride > 1
        self.ff = torch.nn.quantized.FloatFunctional()

    def forward(self, x: Tensor) -> Tensor:
        if self.use_res_connect:
            #return x + self.conv(x)
            return self.ff.add(x, self.conv(x))
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(
        self,
        num_classes: int = 10,
        width_mult: float = 1.0,
        inverted_residual_setting: Optional[List[List[int]]] = None,
        round_nearest: int = 8,
        block: Optional[Callable[..., nn.Module]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        """
        MobileNet V2 main class

        Args:
            num_classes (int): Number of classes
            width_mult (float): Width multiplier - adjusts number of channels in each layer by this amount
            inverted_residual_setting: Network structure
            round_nearest (int): Round the number of channels in each layer to be a multiple of this number
            Set to 1 to turn off rounding
            block: Module specifying inverted residual building block for mobilenet
            norm_layer: Module specifying the normalization layer to use

        """
        super(MobileNetV2, self).__init__()

        if block is None:
            block = InvertedResidual

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        input_channel = 32
        last_channel = 1280

        if inverted_residual_setting is None:
            inverted_residual_setting = [
                # t, c, n, s
                [1, 16, 1, 1],
                [6, 24, 2, 1],  # NOTE: change stride 2 -> 1 for CIFAR10/100
                [6, 32, 3, 2],
                [6, 64, 4, 2],
                [6, 96, 3, 1],
                [6, 160, 3, 2],
                [6, 320, 1, 1],
            ]

        # only check the first element, assuming user knows t,c,n,s are required
        if len(inverted_residual_setting) == 0 or len(inverted_residual_setting[0]) != 4:
            raise ValueError("inverted_residual_setting should be non-empty "
                             "or a 4-element list, got {}".format(inverted_residual_setting))

        # building first layer
        input_channel = _make_divisible(input_channel * width_mult, round_nearest)
        self.last_channel = _make_divisible(last_channel * max(1.0, width_mult), round_nearest)
        features: List[nn.Module] = [ConvBNReLU(1, input_channel, stride=1, norm_layer=norm_layer)]  # NOTE: change stride 2 -> 1 for CIFAR10/100
        # building inverted residual blocks
        for t, c, n, s in inverted_residual_setting:
            output_channel = _make_divisible(c * width_mult, round_nearest)
            for i in range(n):
                stride = s if i == 0 else 1
                features.append(block(input_channel, output_channel, stride, expand_ratio=t, norm_layer=norm_layer))
                input_channel = output_channel
        # building last several layers
        features.append(ConvBNReLU(input_channel, self.last_channel, kernel_size=1, norm_layer=norm_layer))
        # make it nn.Sequential
        self.features = nn.Sequential(*features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.last_channel, num_classes),
        )

        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # This exists since TorchScript doesn't support inheritance, so the superclass method
        # (this one) needs to have a name other than `forward` that can be accessed in a subclass
        x = self.features(x)
        # Cannot use "squeeze" as batch-size can be 1
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


## FashionMNIST Model Quantization

In [ ]:
resnet20_fp_model = CifarResNet(BasicBlock, [3]*3)

resnet20_fp_model.load_state_dict(torch.load("/content/drive/MyDrive/Thesis/ResNet20/FashionMNIST/Train Iteration 3/model/ResNet20_3it_Fashion_93.01acc.pkl",map_location=torch.device('cpu')))

resnet20_model = CifarResNet(BasicBlock, [3]*3)

resnet20_model.load_state_dict(torch.load("/content/drive/MyDrive/Thesis/ResNet20/FashionMNIST/Train Iteration 3/model/ResNet20_3it_Fashion_93.01acc.pkl",map_location=torch.device('cpu')))


resnet20_model = torch.quantization.QuantWrapper(resnet20_model)
B=8
resnet20_model.qconfig = torch.quantization.QConfig(activation= MovingAverageMinMaxObserver.with_args(quant_min=0, quant_max=int(2 ** B - 1), dtype=torch.quint8,
                                                              qscheme=torch.per_tensor_affine, reduce_range=False),
                                                     weight= MovingAverageMinMaxObserver.with_args(quant_min=int(-(2 ** B) / 2), quant_max=int((2 ** B) / 2 - 1), dtype=torch.qint8,
                                                              qscheme=torch.per_tensor_symmetric, reduce_range=False))
torch.quantization.prepare(resnet20_model, inplace=True)

resnet20_model.to("cpu")
test(resnet20_model, fashion_testloader, cuda=False)
resnet20_model.to("cpu")

torch.quantization.convert(resnet20_model, inplace=True)


QuantWrapper(
  (quant): Quantize(scale=tensor([0.0039]), zero_point=tensor([0]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (module): CifarResNet(
    (conv1): QuantizedConv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.010157379321753979, zero_point=130, padding=(1, 1), bias=False)
    (bn1): QuantizedBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): QuantizedConv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.021488258615136147, zero_point=178, padding=(1, 1), bias=False)
        (bn1): QuantizedBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): QuantizedConv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.014824401587247849, zero_point=134, padding=(1, 1), bias=False)
        (bn2): QuantizedBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_runni

In [ ]:
resnet56_fp_model = CifarResNet(BasicBlock,[9]*3)

resnet56_fp_model.load_state_dict(torch.load("/content/drive/MyDrive/Thesis/ResNet56/FashionMNIST/Train Iteration 3/model/ResNet56_3it_Fashion_93.29acc.pkl",map_location=torch.device('cpu')))

resnet56_model = CifarResNet(BasicBlock,[9]*3)

resnet56_model.load_state_dict(torch.load("/content/drive/MyDrive/Thesis/ResNet56/FashionMNIST/Train Iteration 3/model/ResNet56_3it_Fashion_93.29acc.pkl",map_location=torch.device('cpu')))


resnet56_model = torch.quantization.QuantWrapper(resnet56_model)
B=8
resnet56_model.qconfig = torch.quantization.QConfig(activation= MovingAverageMinMaxObserver.with_args(quant_min=0, quant_max=int(2 ** B - 1), dtype=torch.quint8,
                                                              qscheme=torch.per_tensor_affine, reduce_range=False),
                                                     weight= MovingAverageMinMaxObserver.with_args(quant_min=int(-(2 ** B) / 2), quant_max=int((2 ** B) / 2 - 1), dtype=torch.qint8,
                                                              qscheme=torch.per_tensor_symmetric, reduce_range=False))
torch.quantization.prepare(resnet56_model, inplace=True)

resnet56_model.to("cpu")
test(resnet56_model, fashion_testloader, cuda=False)
resnet56_model.to("cpu")

torch.quantization.convert(resnet56_model, inplace=True)

QuantWrapper(
  (quant): Quantize(scale=tensor([0.0039]), zero_point=tensor([0]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (module): CifarResNet(
    (conv1): QuantizedConv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.006637860555201769, zero_point=118, padding=(1, 1), bias=False)
    (bn1): QuantizedBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): QuantizedConv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.006438401993364096, zero_point=156, padding=(1, 1), bias=False)
        (bn1): QuantizedBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): QuantizedConv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), scale=0.00343819591216743, zero_point=127, padding=(1, 1), bias=False)
        (bn2): QuantizedBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [ ]:
mobilenet_fp_model = MobileNetV2(width_mult = 1.4)

mobilenet_fp_model.load_state_dict(torch.load("/content/drive/MyDrive/Thesis/MobileNetv2_x1_4/FashionMNIST/Train Iteration 3/model/MobileNet_3it_Fashion_93.31acc.pkl",map_location=torch.device('cpu')))

mobilenet_model = MobileNetV2(width_mult = 1.4)

mobilenet_model.load_state_dict(torch.load("/content/drive/MyDrive/Thesis/MobileNetv2_x1_4/FashionMNIST/Train Iteration 3/model/MobileNet_3it_Fashion_93.31acc.pkl",map_location=torch.device('cpu')))


mobilenet_model = torch.quantization.QuantWrapper(mobilenet_model)
B=8
mobilenet_model.qconfig = torch.quantization.QConfig(activation= MovingAverageMinMaxObserver.with_args(quant_min=0, quant_max=int(2 ** B - 1), dtype=torch.quint8,
                                                              qscheme=torch.per_tensor_affine, reduce_range=False),
                                                     weight= MovingAverageMinMaxObserver.with_args(quant_min=int(-(2 ** B) / 2), quant_max=int((2 ** B) / 2 - 1), dtype=torch.qint8,
                                                              qscheme=torch.per_tensor_symmetric, reduce_range=False))
torch.quantization.prepare(mobilenet_model, inplace=True)

mobilenet_model.to("cpu")
test(mobilenet_model, fashion_testloader, cuda=False)
mobilenet_model.to("cpu")

torch.quantization.convert(mobilenet_model, inplace=True)

QuantWrapper(
  (quant): Quantize(scale=tensor([0.0039]), zero_point=tensor([0]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (module): MobileNetV2(
    (features): Sequential(
      (0): ConvBNActivation(
        (0): QuantizedConv2d(1, 48, kernel_size=(3, 3), stride=(1, 1), scale=0.00594725925475359, zero_point=137, padding=(1, 1), bias=False)
        (1): QuantizedBatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): QuantizedReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNActivation(
            (0): QuantizedConv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), scale=0.01379318442195654, zero_point=102, padding=(1, 1), groups=48, bias=False)
            (1): QuantizedBatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): QuantizedReLU6(inplace=True)
          )
          (1): QuantizedConv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), scale=0.05

In [ ]:
#ResNet20 DataLoaders

test_20_0 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_20_test_0).type(torch.FloatTensor), torch.from_numpy(y_f_test_set).type(torch.LongTensor))
test_20_0_loader = torch.utils.data.DataLoader(dataset=test_20_0, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_20_1 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_20_test_1).type(torch.FloatTensor), torch.from_numpy(y_f_test_set).type(torch.LongTensor))
test_20_1_loader = torch.utils.data.DataLoader(dataset=test_20_1, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_20_2 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_20_test_2).type(torch.FloatTensor), torch.from_numpy(y_f_test_set).type(torch.LongTensor))
test_20_2_loader = torch.utils.data.DataLoader(dataset=test_20_2, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_20_3 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_20_test_3).type(torch.FloatTensor), torch.from_numpy(y_f_test_set).type(torch.LongTensor))
test_20_3_loader = torch.utils.data.DataLoader(dataset=test_20_3, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)


train_20_0 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_20_train_0).type(torch.FloatTensor), torch.from_numpy(y_f_train_set).type(torch.LongTensor))
train_20_0_loader = torch.utils.data.DataLoader(dataset=train_20_0, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_20_1 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_20_train_1).type(torch.FloatTensor), torch.from_numpy(y_f_train_set).type(torch.LongTensor))
train_20_1_loader = torch.utils.data.DataLoader(dataset=train_20_1, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_20_2 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_20_train_2).type(torch.FloatTensor), torch.from_numpy(y_f_train_set).type(torch.LongTensor))
train_20_2_loader = torch.utils.data.DataLoader(dataset=train_20_2, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_20_3 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_20_train_3).type(torch.FloatTensor), torch.from_numpy(y_f_train_set).type(torch.LongTensor))
train_20_3_loader = torch.utils.data.DataLoader(dataset=train_20_3, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)



In [ ]:
#ResNet56 DataLoaders

test_56_0 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_56_test_0).type(torch.FloatTensor), torch.from_numpy(y_f_test_set).type(torch.LongTensor))
test_56_0_loader = torch.utils.data.DataLoader(dataset=test_56_0, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_56_1 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_56_test_1).type(torch.FloatTensor), torch.from_numpy(y_f_test_set).type(torch.LongTensor))
test_56_1_loader = torch.utils.data.DataLoader(dataset=test_56_1, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_56_2 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_56_test_2).type(torch.FloatTensor), torch.from_numpy(y_f_test_set).type(torch.LongTensor))
test_56_2_loader = torch.utils.data.DataLoader(dataset=test_56_2, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_56_3 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_56_test_3).type(torch.FloatTensor), torch.from_numpy(y_f_test_set).type(torch.LongTensor))
test_56_3_loader = torch.utils.data.DataLoader(dataset=test_56_3, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)


train_56_0 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_56_train_0).type(torch.FloatTensor), torch.from_numpy(y_f_train_set).type(torch.LongTensor))
train_56_0_loader = torch.utils.data.DataLoader(dataset=train_56_0, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_56_1 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_56_train_1).type(torch.FloatTensor), torch.from_numpy(y_f_train_set).type(torch.LongTensor))
train_56_1_loader = torch.utils.data.DataLoader(dataset=train_56_1, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_56_2 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_56_train_2).type(torch.FloatTensor), torch.from_numpy(y_f_train_set).type(torch.LongTensor))
train_56_2_loader = torch.utils.data.DataLoader(dataset=train_56_2, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_56_3 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_56_train_3).type(torch.FloatTensor), torch.from_numpy(y_f_train_set).type(torch.LongTensor))
train_56_3_loader = torch.utils.data.DataLoader(dataset=train_56_3, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)



In [ ]:
#MobileNet Dataset

test_mob_0 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_mob_test_0).type(torch.FloatTensor), torch.from_numpy(y_f_test_set).type(torch.LongTensor))
test_mob_0_loader = torch.utils.data.DataLoader(dataset=test_mob_0, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_mob_1 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_mob_test_1).type(torch.FloatTensor), torch.from_numpy(y_f_test_set).type(torch.LongTensor))
test_mob_1_loader = torch.utils.data.DataLoader(dataset=test_mob_1, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_mob_2 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_mob_test_2).type(torch.FloatTensor), torch.from_numpy(y_f_test_set).type(torch.LongTensor))
test_mob_2_loader = torch.utils.data.DataLoader(dataset=test_mob_2, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

test_mob_3 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_mob_test_3).type(torch.FloatTensor), torch.from_numpy(y_f_test_set).type(torch.LongTensor))
test_mob_3_loader = torch.utils.data.DataLoader(dataset=test_mob_3, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)


train_mob_0 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_mob_train_0).type(torch.FloatTensor), torch.from_numpy(y_f_train_set).type(torch.LongTensor))
train_mob_0_loader = torch.utils.data.DataLoader(dataset=train_mob_0, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_mob_1 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_mob_train_1).type(torch.FloatTensor), torch.from_numpy(y_f_train_set).type(torch.LongTensor))
train_mob_1_loader = torch.utils.data.DataLoader(dataset=train_mob_1, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_mob_2 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_mob_train_2).type(torch.FloatTensor), torch.from_numpy(y_f_train_set).type(torch.LongTensor))
train_mob_2_loader = torch.utils.data.DataLoader(dataset=train_mob_2, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

train_mob_3 = torch.utils.data.TensorDataset(torch.from_numpy(x_f_mob_train_3).type(torch.FloatTensor), torch.from_numpy(y_f_train_set).type(torch.LongTensor))
train_mob_3_loader = torch.utils.data.DataLoader(dataset=train_mob_3, batch_size=64, num_workers=2, pin_memory=True, shuffle=False, collate_fn=custom_collate)

In [ ]:
print("FashionMNIST TEST SETS:")


res20_1, res20_5 = evaluator(resnet20_model,test_20_0_loader)
print("ResNet20 0: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

res20_1, res20_5 = evaluator(resnet20_model,test_20_1_loader)
print("ResNet20 1: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

res20_1, res20_5 = evaluator(resnet20_model,test_20_2_loader)
print("ResNet20 2: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

res20_1, res20_5 = evaluator(resnet20_model,test_20_3_loader)
print("ResNet20 3: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

print("\n")

res56_1, res56_5 = evaluator(resnet56_model,test_56_0_loader)
print("ResNet56 0: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

res56_1, res56_5 = evaluator(resnet56_model,test_56_1_loader)
print("ResNet56 1: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

res56_1, res56_5 = evaluator(resnet56_model,test_56_2_loader)
print("ResNet56 2: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

res56_1, res56_5 = evaluator(resnet56_model,test_56_3_loader)
print("ResNet56 3: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

print("\n")

mob_1, mob_5 = evaluator(mobilenet_model,test_mob_0_loader)
print("MobileNet 0: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))
mob_1, mob_5 = evaluator(mobilenet_model,test_mob_1_loader)
print("MobileNet 1: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))
mob_1, mob_5 = evaluator(mobilenet_model,test_mob_2_loader)
print("MobileNet 2: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))
mob_1, mob_5 = evaluator(mobilenet_model,test_mob_3_loader)
print("MobileNet 3: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))

FashionMNIST TEST SETS:
ResNet20 0: Top_1: 92.50% Top_5:99.75%
ResNet20 1: Top_1: 93.00% Top_5:99.75%
ResNet20 2: Top_1: 92.00% Top_5:99.75%
ResNet20 3: Top_1: 5.50% Top_5:99.75%


ResNet56 0: Top_1: 94.75% Top_5:99.75%
ResNet56 1: Top_1: 94.25% Top_5:99.75%
ResNet56 2: Top_1: 93.75% Top_5:99.75%
ResNet56 3: Top_1: 4.00% Top_5:100.00%


MobileNet 0: Top_1: 93.00% Top_5:99.25%
MobileNet 1: Top_1: 92.75% Top_5:99.75%
MobileNet 2: Top_1: 93.75% Top_5:99.50%
MobileNet 3: Top_1: 3.75% Top_5:99.75%


In [ ]:
print("FashionMNIST TRAIN SETS:")


res20_1, res20_5 = evaluator(resnet20_model,train_20_0_loader)
print("ResNet20 0: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

res20_1, res20_5 = evaluator(resnet20_model,train_20_1_loader)
print("ResNet20 1: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

res20_1, res20_5 = evaluator(resnet20_model,train_20_2_loader)
print("ResNet20 2: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

res20_1, res20_5 = evaluator(resnet20_model,train_20_3_loader)
print("ResNet20 3: Top_1: {}% Top_5:{}%".format(res20_1, res20_5))

print("\n")

res56_1, res56_5 = evaluator(resnet56_model,train_56_0_loader)
print("ResNet56 0: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

res56_1, res56_5 = evaluator(resnet56_model,train_56_1_loader)
print("ResNet56 1: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

res56_1, res56_5 = evaluator(resnet56_model,train_56_2_loader)
print("ResNet56 2: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

res56_1, res56_5 = evaluator(resnet56_model,train_56_3_loader)
print("ResNet56 3: Top_1: {}% Top_5:{}%".format(res56_1, res56_5))

print("\n")

mob_1, mob_5 = evaluator(mobilenet_model,train_mob_0_loader)
print("MobileNet 0: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))
mob_1, mob_5 = evaluator(mobilenet_model,train_mob_1_loader)
print("MobileNet 1: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))
mob_1, mob_5 = evaluator(mobilenet_model,train_mob_2_loader)
print("MobileNet 2: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))
mob_1, mob_5 = evaluator(mobilenet_model,train_mob_3_loader)
print("MobileNet 3: Top_1: {}% Top_5:{}%".format(mob_1, mob_5))
print(test(mobilenet_model,train_mob_3_loader, cuda=False))

FashionMNIST TRAIN SETS:
ResNet20 0: Top_1: 99.00% Top_5:100.00%
ResNet20 1: Top_1: 99.00% Top_5:100.00%
ResNet20 2: Top_1: 99.75% Top_5:100.00%
ResNet20 3: Top_1: 0.75% Top_5:100.00%


ResNet56 0: Top_1: 99.75% Top_5:100.00%
ResNet56 1: Top_1: 99.50% Top_5:100.00%
ResNet56 2: Top_1: 99.75% Top_5:100.00%
ResNet56 3: Top_1: 0.25% Top_5:100.00%


MobileNet 0: Top_1: 99.75% Top_5:100.00%
MobileNet 1: Top_1: 99.75% Top_5:100.00%
MobileNet 2: Top_1: 99.75% Top_5:100.00%
MobileNet 3: Top_1: 0.00% Top_5:100.00%
0.0


In [ ]:
print("FashionMNIST TESTSET EVALUATOR")

res20_top1, res20_top5 = evaluator_testset(resnet20_fp_model, fashion_testloader)
print("ResNet20 Floating-Point Evaluation: Top 1: {}% Top 5: {}%".format(res20_top1, res20_top5))

res20_top1, res20_top5 = evaluator_testset(resnet20_model, fashion_testloader)
print("ResNet20 Quantized Evaluation: Top 1: {}% Top 5: {}%".format(res20_top1, res20_top5))


res56_top1, res56_top5 = evaluator_testset(resnet56_fp_model, fashion_testloader)
print("ResNet56 Floating-Point Evaluation: Top 1: {}% Top 5: {}%".format(res56_top1, res56_top5))

res56_top1, res56_top5 = evaluator_testset(resnet56_model, fashion_testloader)
print("ResNet56 Quantized Evaluation: Top 1: {}% Top 5: {}%".format(res56_top1, res56_top5))


mob_top1, mob_top5 = evaluator_testset(mobilenet_fp_model, fashion_testloader)
print("MobileNet Floating-Point Evaluation: Top 1: {}% Top 5: {}%".format(mob_top1, mob_top5))

mob_top1, mob_top5 = evaluator_testset(mobilenet_model, fashion_testloader)
print("MobileNet Quantized Evaluation: Top 1: {}% Top 5: {}%".format(mob_top1, mob_top5))


FashionMNIST TESTSET EVALUATOR
ResNet20 Floating-Point Evaluation: Top 1: 93.01% Top 5: 99.68%
ResNet20 Quantized Evaluation: Top 1: 92.97% Top 5: 99.66%
ResNet56 Floating-Point Evaluation: Top 1: 93.29% Top 5: 99.75%
ResNet56 Quantized Evaluation: Top 1: 93.12% Top 5: 99.76%
MobileNet Floating-Point Evaluation: Top 1: 93.31% Top 5: 99.44%
MobileNet Quantized Evaluation: Top 1: 93.15% Top 5: 99.45%
